<a href="https://colab.research.google.com/github/rorisDS/workshop_chatbot/blob/main/Workshop_Chatbot_ES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Construyendo un chatbot basado en Inteligencia Artificial

Este notebook ha sido creado para un workshop durante el *Foro Tecnolóxico de Emprego* de 2022 de la Universidad de Vigo por [Víctor Manuel Alonso Rorís](https://es.linkedin.com/in/victor-roris/en) de la empresa [DataSpartan](http://dataspartan.es/)

La presentación y resto de código asociado está disponible en el github: https://github.com/rorisDS/workshop_chatbot

## Qué es un chatbot

Un chatbot es un agente software que permite automatizar conversaciones en lenguaje natural. 

El lenguaje natural es el lenguaje hablado por humanos. Este es desestructurado, por lo tanto difícil de procesar por las máquinas. En la construcción de chatbots se utiliza el área del Procesado de Lenguaje Natural (o NLP por sus siglas en inglés) para interpretar los mensajes. En general, este proceso de interpretación es lo que llamamos NLU (Natural Language Understanding) y que en los chatbots tratan de identificar los componentes del mensaje:

 - *Intents*: Intención del usuario expresada en el mensaje
 - *Entities*: Información adicional del mensaje

A modo general, la arquitectura de un chatbot simple seguiría el siguiente diagrama: 

![arquitectura](https://raw.githubusercontent.com/rorisDS/workshop_chatbot/main/assets/chatbot_architecture.png)

## Implementación

Este notebook tiene un objetivo didáctico. En concreto, aprender los diferentes componentes y ver como se coordinan a fin de automatizar conversaciones muy básicas. Por lo tanto, el código aquí presentado es simple y puede presentar errores. Si estas buscando construir un chatbot orientado a negocio dispones de diferentes librerías especializadas que te pueden servir de base (por ejemplo, [RASA](https://rasa.com/)) 


En este ejemplo vamos a desarrollar un chatbot llamado **Bea** que permita gestionar consultas y peticiones en una cafetería. En concreto:

 - **Consultar el horario**
 - **Consultar el menú**
 - **Hacer una comanda**

La idea es presentar diferentes implementaciones de un chatbot que nos permita profundizar en diferentes tipos de NLUs, desde lo más básico a lo más avanzado. En este sentido presentaremos 3 tipos de componentes NLU:

 - *Basado en patrones*: identificar patrones en los mensajes del usuario 
 - *Basado en similitud de oraciones*: los mensajes se parecen a los esperados para cada intent. 
 - *Basado en Rasa NLU*: modelo NLP para la identificación de `intents` y `entities`



### Instalación de librerias

**NOTA**: Se recomienda configurar el notebook para disponer de una GPU:

```Entorno de ejecución > Cambiar tipo de entorno de ejecución > Acelerador = GPU```

De otra forma debería seguir funcionando, pero los modelos podrían ir más lentos y pueden producirse reinicios de sesión.

In [ ]:
# Libreria para descargar y entrenar state-of-the-art modelos NLP pre-entrenados
!pip install transformers

In [ ]:
# Permitir asincronia en google colab (rasa nlu)
!pip3 install nest_asyncio

In [ ]:
# Version necesaria para rasa nlu
!pip install sanic==21.9.3

In [ ]:
# Version necesaria para rasa nlu
!pip install urllib3==1.26.8

In [ ]:
# Rasa, libreria para el desarrollo de chatbots
!pip install rasa --use-deprecated=legacy-resolver

**NOTA**: Tras instalar estas librerias debes reiniciar el entorno para que estas pasen a ser accesibles en el código:

```Entorno de ejecución > Reiniciar entorno de ejecución```

Tras ello, reanuda la ejecucion de las celdas desde este punto.

In [ ]:
# Necesario, para reinicio
!pip install -U ipython

In [ ]:
# Check if the installation was correct
from urllib3.util.ssl_ import PROTOCOL_TLS

### Capa de BACKEND

Este componente es el encargado de comunicar el chatbot con la API de la cafetería, permitiendo así acceder a los datos (p. ej., almacenados en una base de datos) o ejecutar acciones.   


In [ ]:
# Horario de la cafeteria
timetable_db = [
      {
          "days" : ["Lunes"],
          "timetable" : "08:30-20:00"
      },
      {
          "days" : ["Martes", "Miércoles", "Jueves"],
          "timetable" : "08:00-20:00"
      },
      {
          "days" : ["Viernes"],
          "timetable" : "08:00-17:00"
      }
  ]

# Menú de la cafetería
menu_db = {
    "1" : {
        "item": "Bocadillo de jamón serrano",
        "group": "Bocadillo",
        "cost": 5.30,
        "veg": False,
    },
    "2" : {
        "item": "Bocadillo de queso",
        "group": "Bocadillo",
        "cost": 4.00,
        "veg": True,
    },
    "3" : {
        "item": "Bocadillo de bacon y queso",
        "group": "Bocadillo",
        "cost": 5.00,
        "veg": False,
    },
    "4" : {
        "item": "Pechuga de pollo",
        "group": "Plato combinado",
        "cost": 6.00,
        "veg": True,
    },
    "5" : {
        "item": "Tofu con pure de guisantes",
        "group": "Plato combinado",
        "cost": 7.00,
        "veg": True,
    },
    "6" : {
        "item": "Pimientos rellenos de champiñones",
        "group": "Plato combinado",
        "cost": 6.00,
        "veg": True,
    },
    "7" : {
        "item": "Agua",
        "group": "Bebida",
        "cost": 0.80,
        "veg": None,
    },
    "8" : {
        "item": "Coca-Cola",
        "group": "Bebida",
        "cost": 1.10,
        "veg": None,
    }
}

In [ ]:
import random

class Backend:
  timetable_db = None
  menu_db = None
  order_id = None
  cooking_orders = None

  def __init__(self):
    global timetable_db
    global menu_db 
    self.timetable_db = timetable_db
    self.menu_db = menu_db
    self.order_id = 1
    self.cooking_orders = {}

  # ---- TIMETABLE

  def get_timetable(self) -> list:
    """Devuelve el horario de la cafeteria"""
    return self.timetable_db

  # ---- MENU

  def get_menu(self) -> dict:
    """Recupera el menu de la cafeteria"""
    return self.menu_db

  def get_menu_item(self, item_ids: list) -> list:
    """Recupera los menus que se asocien a los IDs dados"""
    if type(item_ids) != list:
      item_ids = [item_ids]
    return [self.menu_db[item_id] for item_id in item_ids if item_id in self.menu_db]

  # ---- ORDER

  def new_order(self, menus: list) -> str:
    """Registra una comanda y asigna un ID"""
    self.cooking_orders[self.order_id] = menus
    assigned_id = f"BEA{self.order_id}"
    self.order_id += 1
    return assigned_id

  def check_order(self, asked_order_id: str) -> bool:
    """Consulta si la comanda esta disponible para recoger"""
    if asked_order_id not in self.cooking_orders:
      return True
    ready = bool(random.getrandbits(1))
    if ready:
      del self.cooking_orders[asked_order_id]
    return ready

### NLG (Natural Language Generation)

Componente destinado a la generación de lenguaje natural a fin de dar respuesta al usuario humano. 


In [ ]:
class NLG:

  @staticmethod
  def say_hi() -> str:
    return "Hola, mi nombre es Bea.\nSoy el chatbot de tu cafetería de referencia y mi objetivo es ayudarte con tus consultas y peticiones. Solo dime qué quieres que haga por ti."

  @staticmethod
  def say_hi_respond(username: str=None) -> str:
    if username == None:
      username = ""
    return f"Buenas {username}, ¿qué puedo hacer por ti?"

  @staticmethod
  def say_bye() -> str:
    return "Gracias por tu visita, vuelve pronto!"

  @staticmethod
  def say_nounderstand() -> str:
    return "Lo siento no he entendido tu mensaje"

  @staticmethod
  def say_cancel_accept() -> str:
    return "Perfecto, he cancelado tu consulta"

  @staticmethod
  def say_commands() -> str:
    return """Puedes preguntarme los siguientes comandos:
      - Escribe: `horario` para conocer nuestro horario de apertura.
      - Escribe: `menú` para conocer nuestra carta.
      - Escribe: `pedir` para pedir algo de nuestra carta."""

  @staticmethod
  def say_timetable(timetable: dict) -> str:
    if timetable is None:
      return "Actualmente no dispongo de esa información"

    if len(timetable)==0:
      return "La cafetería esta cerrada hasta nuevo aviso"

    timetable_response = ""
    for entry in timetable:
      if len(entry["days"]) == 1:
        timetable_response += f'\nEl {entry["days"][0]} nuestro horario {entry["timetable"]}.'
      else:
        timetable_response += f'\nLos { ", ".join([day for day in entry["days"]][:-1])} y {entry["days"][-1]} nuestro horario es {entry["timetable"]}.'
    return timetable_response[1:] # Eliminamos el primer salto de linea

  @staticmethod
  def say_menu(menu:dict) -> str:  
    menu_response = "Nuestras opciones en la carta son : "

    for item_id in menu:
      menu_option = menu[item_id]
      menu_response += f'\n - {menu_option["item"]} por {menu_option["cost"]} euros. Código de comanda: {item_id}'
    return menu_response

  @staticmethod
  def say_order_question() -> str:
    return "Indica el código(s) de comanda de tu selección (p. ej., '1, 7' para seleccionar la opción 1 y 7):"

  @staticmethod
  def say_complete_order(orders: list, order_id: str) -> str:
    if type(orders) != list:
      orders = [orders]

    # Lista el nombre de los platos seleccionados
    order_names = ""
    if len(orders) == 1:
      order_names = orders[0]["item"]
    else:
      order_names = ", ".join([order["item"] for order in orders][:-1]) 
      order_names += f' y {orders[-1]["item"]}'
    order_response = f"Has seleccionado: {order_names} \n"

    # Calcula el total
    total = sum([float(order["cost"]) for order in orders])
    order_response += f"El total de tu pedido asciende a {total} euros.\n"

    # Identificador de la comanda
    order_response += f"Tu pedido tiene el ID : '{order_id}'.\n"

    order_response += f"Pasa por la barra en un rato a recogerlo!"

    return order_response

  @staticmethod
  def say_order_ready(ready: bool, order_id: str) -> str:
    if ready:
      return f"Tu comanda con id {order_id} está disponible. Pasa por la barra a recogerla."
    return f"Tu comanda con id {order_id} aún está no esta disponible."

### Lógica Conversacional

Componente que gestiona la conversación. Para ello, es el encargado de coordinar a los diferentes componentes y mantener el estado.

In [ ]:
class ConversationManager():
  """
  Componente que gestiona la automatización de la conversación y la lógica
  de interacción con el backend
  """
  
  backend = None
  nlg = None
  nlu_component = None
  close_conversation = None
  order_state = None

  def __init__(self, nlu_component, static_patterns = False):
    
    # Asigna componentes
    self.nlu_component = nlu_component
    self.backend = Backend()
    self.nlg = NLG()

    # La version basada en patrones fija los comandos de entrada
    self.static_patterns = static_patterns
    # La conversacion esta abierta 
    self.close_conversation = False
    # No se inicio el estado de comanda 
    self.order_state = False


  def is_last_message(self):
    """Comprueba si el chatbot da por finalizada la conversacion"""
    return self.close_conversation

  def init_conversation(self):
    """El mensaje con el que el chatbot inicia la conversacion con el usuario"""
    self.close_conversation = False
    response = self.nlg.say_hi()
    if self.static_patterns:
      response += "\n" + self.nlg.say_commands()
    return response

  def _get_entity_values(self, entities, keyword, only_one=False):
    entity_values = [entity["value"] for entity in entities if entity["entity"] == keyword]
    if only_one:
      if len(entity_values)>0:
        return entity_values[0]
      else:
        return None
    return entity_values
    
  
  def logic(self, user_message):
    """Logica que controla el chatbot"""

    if self.order_state:
      # Proceso de solicitar una comanda
      return self._order_logic(user_message)
    else:
      # Logica asociada a la gestion de los comandos generales
      return self._general_logic(user_message)
    
  def _general_logic(self, user_message):
    """Logica asociada a la gestion de los comandos generales"""

    # Llama al NLU para interpretar el mensaje
    intent, entities = self.nlu_component(user_message)

    if intent == "greet":
      # Extrae el username de las entities (de existir)
      username = self._get_entity_values(entities=entities,
                                         keyword="username",
                                         only_one=True)
      response = self.nlg.say_hi_respond(username)
      return response

    if intent == "timetable":
      timetable_data = self.backend.get_timetable()
      response = self.nlg.say_timetable(timetable_data)
      return response

    if intent == "menu":
      menu_data = self.backend.get_menu()
      response = self.nlg.say_menu(menu_data)
      return response

    if intent == "order":
      # Cambia el estado del gestor de conversacion (se inicia la comanda)
      self.order_state = True
      response = self.nlg.say_order_question()
      return response

    if intent == "check_order":
      # Extrae el username de las entities (de existir)
      asked_order_id = self._get_entity_values(entities=entities,
                                         keyword="asked_order_id",
                                         only_one=True)
      ready = self.backend.check_order(asked_order_id=asked_order_id)
      response = self.nlg.say_order_ready(ready, asked_order_id)
      return response

    if intent == "bye":
      # Cambia el estado del gestor de conversacion (se cierra la comunicacion)
      self.close_conversation = True
      response = self.nlg.say_bye()
      return response
  
    return self.nlg.say_nounderstand()

  def _order_logic(self, user_message):
    """Logica asociada a la seleccion de la comanda"""

    user_message = user_message.replace(" y ", ",")
    possible_item_id = [pid.strip() for pid in user_message.split(",")]
    menu_item = self.backend.get_menu_item(possible_item_id)

    if len(menu_item)>0:
      # Cambia el estado del gestor de conversacion (se acabo la comanda)
      self.order_state = False
      assigned_order_id = self.backend.new_order(menu_item)
      response = self.nlg.say_complete_order(menu_item, assigned_order_id)
      return response
    else:
      # El usuario escribio algo q no es una comand
      intent, entities = self.nlu_component(user_message)

      if intent == "menu":
        menu_data = self.backend.get_menu()
        response = self.nlg.say_menu(menu_data)
        response += f"\n {self.nlg.say_order_question()}"
        return response

      if intent == "cancel":
        # Cambia el estado del gestor de conversacion (se cierra la comanda)
        self.order_state = False
        response = self.nlg.say_cancel_accept()
        return response

      if intent == "bye":
        # Cambia el estado del gestor de conversacion (se cierra la comunicacion)
        self.close_conversation = True
        response = self.nlg.say_bye()
        return response
  
      return self.nlg.say_nounderstand()

### Canal de comunicación

Componente encargado de ofrecer una vía de interacción con el usuario.

In [ ]:
def channel(conversation_manager):

    # Chatbot inicia conversacion?
    initial_sms = conversation_manager.init_conversation()
    if len(initial_sms)>0:
        print(f"Bea: {initial_sms} \n")

    while True:
        text = str(input('Yo: '))

        response = conversation_manager.logic(text)
        print(f"Bea: {response} \n")

        if conversation_manager.is_last_message():
          break


### Implementaciones de NLU

Este componente es el que interpreta los mensajes del usuario en Lenguaje Natural y extrae el *Intent* y las *Entities*.

#### Basado en patrones

En esta implementación trataremos de identificar de forma manual alguno de los términos más frecuentes entre los posibles mensajes de cada `intent`. Al mismo tiempo, tenemos que tener en cuenta que no sean términos comunes en el resto de `intents`.

De esta forma podremos identificar con precisión el `intent` asociado a cada mensaje por la presencia o ausencia de cada uno de estos términos clave.

Este principio es el mismo que usamos en [`TF-IDF` (term frequency-inverse document frequency)](https://monkeylearn.com/blog/what-is-tf-idf). El cual sería factible en caso de contar con una cantidad elevada de ejemplos etiquetados. 

![TF-IDF](https://miro.medium.com/max/943/1*HZvxT29V9B4HxT2wx8M4XQ.png)

In [ ]:
def check_some_expression_in_sms(expresions, user_message):
  """Comprueba si alguna de las expresiones dadas estan en el mensaje de entrada"""
  if len([expresion for expresion in expresions if expresion in user_message.lower()])>0:
    return True
  return False

def patternBasedNLU(user_message):
  """
  Componente NLU que pretende identificar el INTENT en base a `palabras` 
  presentes en el mensaje.
  Por ejemplo: si la palabra `horario` esta en el mensaje la intencion del
  usuario probablemente sea consultar el horario de la cafeteria.   
  """

  # - Identifica si el mensaje es relativo al intent : timetable
  timetable_patterns = ["horario", "hora", "cierre", "abrir"]
  if check_some_expression_in_sms(timetable_patterns, user_message):
    return "timetable", None
  
  # - Identifica si el mensaje es relativo al intent : menu
  menu_patterns = ["menú", "menu"] # Rellenar!!!
  if check_some_expression_in_sms(menu_patterns, user_message):
    return "menu", None
  
  # - Identifica si el mensaje es relativo al intent : order
  order_patterns = ["pedir"] # Rellenar!!!
  if check_some_expression_in_sms(order_patterns, user_message):
    return "order", None
  
  # - Identifica si el mensaje es relativo al intent : cancel
  cancel_patterns = ["cancela"] # Rellenar!!!
  if check_some_expression_in_sms(cancel_patterns, user_message):
    return "cancel", None
  
  # - Identifica si el mensaje es relativo al intent : bye
  bye_patterns = ["adios"] # Rellenar!!!
  if check_some_expression_in_sms(bye_patterns, user_message):
    return "bye", None

  return "no_understand", None

In [ ]:
cm = ConversationManager(nlu_component=patternBasedNLU, static_patterns=True)
channel(cm)

Bea: Hola, mi nombre es Bea.
Soy el chatbot de tu cafetería de referencia y mi objetivo es ayudarte con tus consultas y peticiones. Solo dime qué quieres que haga por ti.
Puedes preguntarme los siguientes comandos:
      - Escribe: `horario` para conocer nuestro horario de apertura.
      - Escribe: `menu` para conocer nuestra carta.
      - Escribe: `pedir` para pedir algo de nuestra carta. 

Yo: horario
Bea: El Lunes nuestro horario 08:30-20:00.
Los Martes, Miércoles y Jueves nuestro horario es 08:00-20:00.
El Viernes nuestro horario 08:00-17:00. 

Yo: menu
Bea: Nuestras opciones en la carta son : 
 - Bocadillo de jamón serrano por 5.3 euros. Código de comanda: 1
 - Bocadillo de queso por 4.0 euros. Código de comanda: 2
 - Bocadillo de bacon y queso por 5.0 euros. Código de comanda: 3
 - Pechuga de pollo por 6.0 euros. Código de comanda: 4
 - Tofu con pure de guisantes por 7.0 euros. Código de comanda: 5
 - Pimientos rellenos de champiñones por 6.0 euros. Código de comanda: 6
 - Agua

#### Basado en similitud de oraciones

En este caso buscamos la similitud semántica entre oraciones, ¿dicen algo similar aún escritas de forma diferente?



##### Sentence Similarity

Para calcular cómo de similares son oraciones utilizaremos una red neuronal de NLP que nos permite convertir textos en embeddings. Un *embedding* es un vector matemático (ej. `[0.23824, 0.38510, 1.05822]`) que representa el texto como un punto en un espacio multidimensional. 

Por ejemplo, en la siguiente imagen cada palabra es convertida a embedding y, tras reducir su dimensión, representada en un grafo 2D.  

![alt text](https://i.stack.imgur.com/nlEz7.png)

En este caso concreto vamos a utilizar el modelo `hiiamsid/sentence_similarity_spanish_es` disponible a través de HuggingFace:
https://huggingface.co/hiiamsid/sentence_similarity_spanish_es

Primero cargamos el tokenizador (que convierte el texto a valores válidos de entrada al modelo) y el modelo (que convierte el texto tokenizado a embeddings de salida)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Cargamos el tokenizador y el model de HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('hiiamsid/sentence_similarity_spanish_es')
model = AutoModel.from_pretrained('hiiamsid/sentence_similarity_spanish_es')

Funciones para obtener el embedding de una oración usando el tokenizador y el modelo

In [ ]:
# Mean Pooling - Calcula el embedding usando la attention mask para corregir la media
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] # El primer elemento contiene todos los embeddings de los tokens
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Obtiene el embedding de una frase
def sentence_embedding(sentence: str):

  # Tokeniza la frase
  encoded_input = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt')

  # Usa el modelo para predecir los embeddings de todos los tokens de la frase
  with torch.no_grad():
      model_output = model(**encoded_input)

  # Calcula el embedding de la frase completa
  return mean_pooling(model_output, encoded_input['attention_mask'])

Probamos que funciona

In [ ]:
sentence = "Hola mundo"
emb = sentence_embedding(sentence)

print(f"El vector/embedding de salida tiene la forma: {list(emb.shape)}\n")

# Representacion controlada para evitar visualizar un vector de grandes proporciones por pantalla
print("Embedding : [", list(emb[0])[0].item(), ", ", list(emb[0])[1].item(), ", ", list(emb[0])[2].item(), ", ..., ", list(emb[0])[-1].item(), "]")

El vector/embedding de salida tiene la forma: [1, 768]

Embedding : [ 0.8512957096099854 ,  -0.3589854836463928 ,  0.15464350581169128 , ...,  -0.40280669927597046 ]




---
Una vez disponemos de los embeddings podemos aplicar algún método mátemático para conocer la distancia o similitud entre los vectores.

En este caso, vamos a aplicar [similitud del coseno](https://towardsdatascience.com/understanding-cosine-similarity-and-its-application-fd42f585296a):

![cosine_similarity_func](https://miro.medium.com/max/494/1*EoRUdEW02wzbLCT0VuTzXQ.png)

Para su calculo usaremos directamente la librería `sklearn`.




In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def sentence_similarity(sentence1, sentence2):
  """Calcula la similitud de dos oraciones"""

  # Calculamos los embeddings
  if type(sentence1) == str:
    sentence1 = sentence_embedding(sentence1)
  if type(sentence2) == str:
    sentence2 = sentence_embedding(sentence2)

  # Calculamos la similitud del coseno basada en los embeddings
  return cosine_similarity(sentence1, sentence2).item()

Visualizamos un ejemplo de calculo de similitud entre frases. 

In [ ]:
# Frase de referencia
reference = "Esa es una persona feliz"

# Frases para comparar
sentence1 = "Ese es un perro feliz"
sentence2 = "Esa es una persona muy feliz"
sentence3 = "Hoy es un día soleado"

print(f"Oración de referencia : {reference} \n")
print(f"- {sentence1} \t ({sentence_similarity(reference, sentence1)})")
print(f"- {sentence2} \t ({sentence_similarity(reference, sentence2)})")
print(f"- {sentence3} \t ({sentence_similarity(reference, sentence3)})")

Oración de referencia : Esa es una persona feliz 

- Ese es un perro feliz 	 (0.4153728485107422)
- Esa es una persona muy feliz 	 (0.9393529891967773)
- Hoy es un día soleado 	 (0.18199917674064636)


##### Ejemplos de INTENTS



A continuación, definimos algunos ejemplos de mensajes esperados por cada intent. Para ello, seguiremos el mismo formato que se define en [RASA](https://rasa.com/docs/rasa/training-data-format/). 

In [ ]:
text = """
nlu:

- intent: bye
  examples: |
    - adios
    - bye
    - chao

- intent: timetable
  examples: |
    - que horario teneis
    - que horas estais abiertos
    - a que hora cerrais
    - estais abiertos ahora
    - abris por la mañana
    - abris por la tarde
    - abris por la noche
    - hay servicio hoy
    - seguireis dando servicio en una hora
    - cuando abris
    - esta semana esta la cafeteria en servicio
    - horario de apertura
  
- intent: menu
  examples: |
    - enseñame el menu de la cafeteria
    # Escribe aqui ejemplos para solicitar visualizar el menu


- intent: order
  examples: |
    - quisiera hacer un pedido
    # Escribe aqui ejemplos para solicitar lanzar comandas

- intent: cancel
  examples: |
    - cancela el proceso actual
    # Escribe aqui ejemplos para cancelar un proceso que se ha iniciado

"""

# %store text > nlu.yml
with open('nlu.yml', 'w') as f:
    f.write(text)

##### NLU basado en Sentence Similarity 

Finalmente, implementamos un componente NLU que aplique sentence similarity a los mensajes de entrada contra cada uno de los mensajes de ejemplo, hasta identificar el *intent* mas probable. 

In [ ]:
import yaml
import numpy as np

class SentenceSimilarityBasedNLU:
  similarity_threshold = None

  def __init__(self, nlu_input_filepath = "./nlu.yml",
               similarity_threshold:float = 0.2):
    self.similarity_threshold = similarity_threshold
    self.nlu_input_filepath = nlu_input_filepath
    self.nlu_inputs = self.load_nlu_inputs()
  
  def load_nlu_inputs(self) -> dict:
    """Carga los ejemplos de NLU"""
    with open(self.nlu_input_filepath, 'r') as stream:
        try:
            data = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print("The NLU file is not properly formated. Review it before continuing. ")
            print("Exception: ", exc)
            raise exc
        intents = {}
        for entry in data['nlu']:
          intent = entry['intent']
          examples = [intent_ex.replace("-","").strip() for intent_ex in entry['examples'].splitlines()]
          intents[intent] = [sentence_embedding(example) for example in examples]
        return intents

  def __call__(self, user_message):
    max_sim_score = -1
    max_sim_intent = None

    # Calcula el embedding del mensaje de entrada
    user_message_emb = sentence_embedding(user_message)

    # Recorre cada intent 
    for intent in self.nlu_inputs:
      # Recorre cada ejemplo del intent
      for example in self.nlu_inputs[intent]:

        # Calcula similitud entre el mensaje del usuario y el ejemplo del intent
        similarity = sentence_similarity(user_message_emb, example)

        if similarity > 0.98:
          return intent, None

        # Similitud superior al umbral y a la ultima maxima similitud
        if similarity > self.similarity_threshold and similarity > max_sim_score :
          max_sim_score = similarity
          max_sim_intent = intent
                
    return max_sim_intent, None

Lanzamos el chatbot

In [ ]:
cm = ConversationManager(nlu_component=SentenceSimilarityBasedNLU())
channel(cm)

Bea: Hola, mi nombre es Bea.
Soy el chatbot de tu cafetería de referencia y mi objetivo es ayudarte con tus consultas y peticiones. Solo dime qué quieres que haga por ti. 

Yo: cuando estais abiertos
Bea: El Lunes nuestro horario 08:30-20:00.
Los Martes, Miércoles y Jueves nuestro horario es 08:00-20:00.
El Viernes nuestro horario 08:00-17:00. 

Yo: muestrame el menu de la cafeteria
Bea: Nuestras opciones en la carta son : 
 - Bocadillo de jamón serrano por 5.3 euros. Código de comanda: 1
 - Bocadillo de queso por 4.0 euros. Código de comanda: 2
 - Bocadillo de bacon y queso por 5.0 euros. Código de comanda: 3
 - Pechuga de pollo por 6.0 euros. Código de comanda: 4
 - Tofu con pure de guisantes por 7.0 euros. Código de comanda: 5
 - Pimientos rellenos de champiñones por 6.0 euros. Código de comanda: 6
 - Agua por 0.8 euros. Código de comanda: 7
 - Coca-Cola por 1.1 euros. Código de comanda: 8 

Yo: hacer un pedido
Bea: Indica el código(s) de comanda de tu selección (p. ej., '1, 7' para

#### Basado en Rasa NLU




En este caso utilizamos el componente NLU de la librería [Rasa](https://rasa.com/docs/rasa/).

Este componente utiliza algoritmos propios de IA para la identificación de *Intents* y *Entitities*

##### Ejemplos de INTENTS y ENTITIES

Los ejemplos dados sirven para entrenar el componente. Para que el algoritmo sepa interpretar los datos, estos deben seguir un [formato específico](https://rasa.com/docs/rasa/training-data-format/) y ser exportados en un fichero YAML.

Además, para afinar el NLU es posible [incluír datos de entrenamiento específicos como sinónimos, tablas de *lookup* o expresiones regulares](https://rasa.com/docs/rasa/nlu-training-data/).

En este caso reutilizaremos el `nlu.yml` creado en la sección anterior ya que incluía diferentes ejemplos siguiendo ya el formato de Rasa. A estos ejemplos ya existentes añadiremos (append) nuevos ejemplos con definición de *Entities*.

In [ ]:
text = """
- intent: greet
  examples: |
    - hey
    - hola
    - buenas
    - hola Bea
    - buenos dias
    - buenas tardes
    - buenas noches
    - un placer conocerte Bea
    - que casualidad, yo también me llamo [Bea](username)
    - hola Bea, soy [Victor](username)
    - mi nombre es [Almudena](username), encantada de conocerte
    - buenas, me llamo [Sonia](username)
    - soy [Alberto](username)
    - llámame [Maria](username)
    - hola Bea, soy [Juan](username)

- lookup: username
  examples: |
    - Alejandra
    - Sergio
    - Valentina
    # Escribe mas posibles nombre    


- intent: check_order
  examples: |
    - mi pedido [BEA1001](asked_order_id) esta disponible
    - puedo pasar a recoger la comanda [BEA1](asked_order_id)
    - está [BEA85](asked_order_id) listo?
    - la orden [BEA092](asked_order_id) esta en la barra?
    - [BEA77](asked_order_id) está cocinado?
    - tenéis ya por ahí el menú [BEA254](asked_order_id)    

- regex: asked_order_id
  examples: |
    - BEA\d{1,10}

"""

with open('nlu.yml', 'a') as f:
    f.write(text)

##### Configuración Rasa NLU

La configuración de Rasa NLU se basa en pipelines. Es decir, en la definición de los componentes (y sus configuraciones) y la secuencia de ejecución para llegar a la identificación de *intents* y *entities*. 

Los [componentes del pipeline](https://rasa.com/docs/rasa/components/#languagemodelfeaturizer) permiten realizar diferentes análisis (vectorización, tokenización, clasificación, etc.) y sus resultados sirven de entrada al siguiente componente. Además, es posible definir componentes propios.

En el caso del componente principal el que realiza el paso de clasificar los *intents* y *entities*, usaremos el [*DIET Classifier*](https://www.youtube.com/watch?v=vWStcJDuOUk). Este modelo alcanza el *State of The Art* en la actualidad (aka, principios de 2022).

![DIET Classifier](https://miro.medium.com/max/1400/1*Y_sBnCaJyo4poREWdBS3Wg.gif)

A modo de ejemplo, la configuración propuesta es:

In [ ]:
text = """
recipe: default.v1

language: es

pipeline:
  - name: WhitespaceTokenizer
  - name: LanguageModelFeaturizer
    model_weights: "dccuchile/bert-base-spanish-wwm-cased"
    model_name: "bert"
  - name: CountVectorsFeaturizer
  - name: LexicalSyntacticFeaturizer
  - name: RegexFeaturizer
  - name: DIETClassifier
    random_seed: 42
    use_masked_language_model: True
    epochs: 100
    number_of_transformer_layer: 4
    transformer_size: 256
    drop_rate: 0.2
    weight_sparsity: 0.7
  - name: EntitySynonymMapper
"""

# %store text > config.yml
with open('rasa_config.yml', 'w') as f:
    f.write(text)

In [ ]:
from rasa import model_training
model_path = model_training.train_nlu(
    config="./rasa_config.yml",
    nlu_data="./nlu.yml",
    output="./rasa_models/"
  )

##### NLU basado en RASA NLU 

Finalmente, implementamos un componente NLU que aplique RASA NLU a los mensajes de entrada  pare identificar el *intent* y *entities* mas probable. 

In [ ]:
# Para usar asincronia en Google Colab
import nest_asyncio

nest_asyncio.apply()
print("Event loop ready.")

Event loop ready.


In [ ]:
import rasa
import asyncio

class RasaBasedNLU:

  def __init__(self, model_path = "./rasa_model",
               intent_threshold:float = 0.4):    
    self.agent = asyncio.run(rasa.core.agent.load_agent(model_path=model_path))
    self.intent_threshold = intent_threshold
  

  def __call__(self, user_message):
    
    # Run NLU model
    result = asyncio.run(self.agent.parse_message(user_message))

    # Intent
    if result["intent"]["confidence"]> self.intent_threshold:
        return result["intent"]["name"], result["entities"]

    return "not_found", None

In [ ]:
cm = ConversationManager(nlu_component=RasaBasedNLU(model_path=model_path))

/usr/local/lib/python3.7/dist-packages/rasa/shared/core/slot_mappings.py:217: UserWarning: Slot auto-fill has been removed in 3.0 and replaced with a new explicit mechanism to set slots. Please refer to https://rasa.com/docs/rasa/domain#slots to learn more.
  UserWarning,
Some layers from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly ini

In [ ]:
channel(cm)

Bea: Hola, mi nombre es Bea.
Soy el chatbot de tu cafetería de referencia y mi objetivo es ayudarte con tus consultas y peticiones. Solo dime qué quieres que haga por ti. 

Yo: Hola Bea, soy Victor
Bea: Buenas Victor, ¿qué puedo hacer por ti? 

Yo: Me gustaria saber si estais abiertos
Bea: El Lunes nuestro horario 08:30-20:00.
Los Martes, Miércoles y Jueves nuestro horario es 08:00-20:00.
El Viernes nuestro horario 08:00-17:00. 

Yo: Muestrame la carta
Bea: Nuestras opciones en la carta son : 
 - Bocadillo de jamón serrano por 5.3 euros. Código de comanda: 1
 - Bocadillo de queso por 4.0 euros. Código de comanda: 2
 - Bocadillo de bacon y queso por 5.0 euros. Código de comanda: 3
 - Pechuga de pollo por 6.0 euros. Código de comanda: 4
 - Tofu con pure de guisantes por 7.0 euros. Código de comanda: 5
 - Pimientos rellenos de champiñones por 6.0 euros. Código de comanda: 6
 - Agua por 0.8 euros. Código de comanda: 7
 - Coca-Cola por 1.1 euros. Código de comanda: 8 

Yo: Quisiera hacer un

## Conclusiones



Un chatbot es un proceso lógico basado en la interacción y coordinación de diferentes componentes. Estos pueden variar de lo más complejo a lo más simple y los resultados varian en consonancia.

En este notebook hemos visto un ejemplo muy básico de chatbot pero que nos ofrece un primer acercamiento tanto al ámbito de los chatbots como a NLP. 

## Ejercicios

¿Cómo harías para incluir la opción de consultar la carta pero mostrando solo la opción vegetariana? ¿Un nuevo *intent*?¿o *entity* en el *intent* ya existente? 

¿Cómo se podría consultar el horario de un día específico?¿Cómo hacer para informar si en el momento de la consulta la cafetería esta abierta o no?

¿Y la opción de consultar por tipo de plato: bocadillos, platos combinados, bebidas, etc.? Piensa que el menú puede cambiar y ampliar la oferta.

¿Sería posible hacer que se pueda pedir la comanda por el nombre de los platos? En caso posible (que lo es), ¿qué pasaría cuando el nombre que da el usuario no sea exactamente igual al de la carta?¿Podrías solucionarlo en el NLU (ver [sinónimos](https://rasa.com/docs/rasa/nlu-training-data#synonyms))?

¿Crees que podrías hacer tu propio chatbot? Anímate.

Cualquier duda o comentario estoy encantado de contestarte: [linkedin](https://www.linkedin.com/in/victor-roris)